In [1]:
import numpy as np
import pickle
from learnable_crf import LearnableCrf
from lib import *
from scipy.special import expit as sigmoid

In [2]:
r = 50
with open('cache/df_train.{}.pickle'.format(r), mode='rb') as h:
    df = pickle.load(h)
leaves = np.nonzero([x[0] in x[1] for x in zip(df['label'], df['pseudo_label'])])[0]
Y_train = df['label'][leaves]
with open('cache/df_val_test.pickle', mode='rb') as h:
    df_val, df_test = pickle.load(h)
Y_val = df_val['label']
Y_test = df_test['label']
with open('cache/hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)
state_space = hex_data['state_space']
# state_space = filter(lambda x: x[:20].any(), state_space)

In [3]:
model_Phi_train = np.load('results/caffe_independent/iter_Phi_train.{}.npy'.format(r))[:, leaves]
model_Phi_val = np.load('results/caffe_independent/iter_Phi_val.{}.npy'.format(r))
model_Phi_test = np.load('results/caffe_independent/iter_Phi_test.{}.npy'.format(r))
M = len(model_Phi_train)

In [ ]:
def raw_crf(Phi):
    def step(phi):
        scores = map(lambda s: phi[s].sum(), state_space)
        return np.vstack(tuple(state_space[np.argsort(scores)[i]] for i in range(-5, 0)))
#         return state_space[np.argmax(scores)]
    return np.array(map(step, Phi), dtype=bool)

In [ ]:
# [top5_accuracy(model_Phi_val[i], Y_val, lim_states=False) for i in range(0, M)]
[top5_accuracy(raw_crf(model_Phi_val[i]), Y_val) for i in range(0, M)]

In [ ]:
# [top5_accuracy(model_Phi_test[i], Y_test, lim_states=False) for i in range(0, M)]
[top5_accuracy(raw_crf(model_Phi_test[i]), Y_test) for i in range(0, M)]

In [4]:
model_Phi_train = sigmoid(model_Phi_train)
model_Phi_val = sigmoid(model_Phi_val)
model_Phi_test = sigmoid(model_Phi_test)
lcrf = [LearnableCrf(model_Phi_train[i], Y_train) for i in range(0, M)]
[get_accuracy(lcrf[i].predict(model_Phi_val[i]), Y_val) for i in range(0, M)]

[0.39667458432304037,
 0.43646080760095013,
 0.4667458432304038,
 0.4637767220902613,
 0.4649643705463183,
 0.4655581947743468,
 0.4655581947743468,
 0.4655581947743468,
 0.4655581947743468,
 0.4655581947743468]

In [5]:
[get_accuracy(lcrf[i].predict(model_Phi_test[i]), Y_test) for i in range(0, M)]

[0.3812351543942993,
 0.43764845605700714,
 0.46140142517814725,
 0.46140142517814725,
 0.46199524940617576,
 0.4637767220902613,
 0.4643705463182898,
 0.4643705463182898,
 0.4643705463182898,
 0.4643705463182898]